# Florida UO2 ThO2 Mixture Assignment: 30Oct

In [ ]:
%matplotlib inline
import openmc
import math

In [34]:
while True:
    try:
        print('Please select whether you want to do one run with user inputs (1) or a continuous run (2).')
        userInputTF = int(input())
    except ValueError:
        print("Please type in a valid number (1 or 2)")
        continue
    if userInputTF not in (1,2):
        print("Please type in a valid entry (1 or 2)")
        continue
    else:
        break
if userInputTF == 1: 
    print("Single Run, User Input")
elif userInputTF == 2:
    print("Continuous Run (Currently not supported)")

Please select whether you want to do one run with user inputs (1) or a continuous run (2).


 1


Single Run, User Input


In [39]:
if userInputTF == 1:
    while True:
        try:
            UO2MassRho = float(input('Desired UO2 Mass Density [g/cc]? (Press Enter key for default for default value of 10.45 g/cc)') or 10.45)
        except ValueError:
            print("Please type in a valid number (no characters)")
            continue
        else:
            break
    while True:
        try:
            UO2Enrichment = float(input('UO2 Enrichment [g/cc]? (Press Enter key for default for default value of 10.45 g/cc)') or 10.45)
        except ValueError:
            print("Please type in a valid number (no characters)")
            continue
        else:
            break
    while True:
        try:
            UO2MassRho = float(input('Desired UO2 Mass Density [g/cc]? (Press Enter key for default for default value of 10.45 g/cc)') or 10.45)
        except ValueError:
            print("Please type in a valid number (no characters)")
            continue
        else:
            break
    print('UO2 Mass Density: ', UO2MassRho, 'g/cc')
    print('UO2 Enrichment: ', UO2Enrichment, 'g/cc')
    print('Fuel UO2 Percent: ', pctLEU, 'g/cc')
    print('Fuel Thorium Percent: ', 1 - pctLEU, 'g/cc')


Desired UO2 Mass Density [g/cc]? (Press Enter key for default for default value of 10.45 g/cc) 


UO2 Mass Density:  10.45


## Calculate Weight Percentages

In [ ]:
# Constants - via PNNL Mat'l Comp., Rev-2
ThO2OAtomRho = 0.044247 # Oxygen Atom Density in ThO2 [atom/b-cm]
ThO2ThAtomRho = 0.022124 # Thorium Atom Density in ThO2 [atom/b-cm]
ThMolW = 232. # Molecular Weight of Th [g/mol]
OMolW = 16. # Molecular Weight of O [g/mol]
ThO2MolW = ThMolW + 2*OMolW # Molecular Weight of ThO2 [g/mol]
U235MolW = 235. # Molecular Weight of U-235 [g/mol]
U238MolW = 238. # Molecular Weight of U-238 [g/mol]
ThO2MassRho = 9.7 # Mass Density of ThO2 [g/cm^3]
graphMassRho = 2.26 # Mass Density of Graphite [g/cm^3]
he_CoolMassRho = 0.000178 # Mass Density of He @ 293K [g/cm^3]
avo = 0.6022 # Avogadro's Number

# Results of Constants and Input Parameters
# Weight percent of Th in LEU mixture
pctTh = 1 - pctLEU 
# Total Mixture Density
UThMixMassRho = (pctLEU/UO2MassRho + pctTh/ThO2MassRho)**-1 # [g/cm]
# UO2 Atom Densities
UO2UMolW = ((UO2Enrichment/U235MolW) + ((1 - UO2Enrichment)/U238MolW))**-1 # Molecular Weight of U in UO2 [g/mol]
UO2MolW = UO2UMolW + 2*16 # Molecular Weight of UO2 [g/mol]
UO2OAtomRho = (UO2MassRho/UO2MolW)*2*avo # O Atom Density in UO2 [atom/b-cm]
UO2U235AtomRho = (UO2Enrichment*UO2MassRho*(UO2UMolW/UO2MolW)/U235MolW)*avo # U-235 Atom Density in UO2 [atom/b-cm] 
UO2U238AtomRho = ((1 - UO2Enrichment)*UO2MassRho*(UO2UMolW/UO2MolW)/U238MolW)*avo # U-238 Atom Density in UO2 [atom/b-cm]
UO2U235AtomRhoFrac = UO2U235AtomRho/(UO2U235AtomRho+UO2U238AtomRho)
UO2U238AtomRhoFrac = UO2U238AtomRho/(UO2U235AtomRho+UO2U238AtomRho)
# UO2 ThO2 Mixture Atom Densities
UThMixOAtomRho = (pctTh*UThMixMassRho/ThO2MolW)*2*avo + (pctLEU*UThMixMassRho/UO2MolW)*2*avo # Oxygen Atom Density in Fuel Mixture [atom/b-cm]
UThMixThAtomRho = (pctTh*UThMixMassRho/ThO2MolW)*avo # Uranium Atom Density in Fuel Mixture [atom/b-cm] 
UThMixU235AtomRho = (pctLEU*UThMixMassRho/UO2MolW)*avo*UO2U235AtomRhoFrac # Uranium Atom Density in Fuel Mixture [atom/b-cm]
UThMixU238AtomRho = (pctLEU*UThMixMassRho/UO2MolW)*avo*UO2U238AtomRhoFrac # Uranium Atom Density in Fuel Mixture [atom/b-cm]
# UO2 ThO2 Mixture Atom Fractions
# Normalized to 1
UThMixOAtomFrac = UThMixOAtomRho/(UThMixOAtomRho + UThMixThAtomRho + UThMixU235AtomRho + UThMixU238AtomRho) # Oxygen Atom Fraction in Fuel Mixture [atom/b-cm]
UThMixThAtomFrac = UThMixThAtomRho/(UThMixOAtomRho + UThMixThAtomRho + UThMixU235AtomRho + UThMixU238AtomRho) # Uranium Atom Fraction in Fuel Mixture [atom/b-cm] 
UThMixU235AtomFrac = UThMixU235AtomRho/(UThMixOAtomRho + UThMixThAtomRho + UThMixU235AtomRho + UThMixU238AtomRho) # Uranium Atom Fraction in Fuel Mixture [atom/b-cm]
UThMixU238AtomFrac = UThMixU238AtomRho/(UThMixOAtomRho + UThMixThAtomRho + UThMixU235AtomRho + UThMixU238AtomRho) # Uranium Atom Density in Fuel Mixture [atom/b-cm]
# Normalized to 3
UThMixOAtomFrac3 = UThMixOAtomFrac*3 # Oxygen Atom Fraction in Fuel Mixture [atom/b-cm]
UThMixU235AtomFrac3 = UThMixU235AtomFrac*3 # Uranium Atom Fraction in Fuel Mixture [atom/b-cm] 
UThMixU238AtomFrac3 = UThMixU238AtomFrac*3 # Uranium Atom Fraction in Fuel Mixture [atom/b-cm]
UThMixThAtomFrac3 = UThMixThAtomFrac*3 # Uranium Atom Density in Fuel Mixture [atom/b-cm]

## Calculate Vol %

In [ ]:
# Constants
vF_Pebbles_Core = 2./3 # Volume Fraction of Pebbles in Core (Max packing factor)
vF_TRISO_Pebbles = 0.4 # Volume Fraction of TRISO in Pebbles
vF_Fuel_TRISO = 0.15 # Volume Fraction of Fuel in TRISO
fuelVolFrac = vF_Pebbles_Core*vF_TRISO_Pebbles*vF_Fuel_TRISO # Volume Fraction of Fuel
graphVolFrac = vF_Pebbles_Core*(1 - (vF_TRISO_Pebbles*vF_Fuel_TRISO)) # Total Volume Fraction of Graphite
hel_CoolVolFrac = 1 - (fuelVolFrac + graphVolFrac) # Total Volume Fraction of Helium
totalVolFrac = fuelVolFrac + graphVolFrac + hel_CoolVolFrac # Total Volume

# Parameters
# Cube
matCubeL = 1 # [cm]
cubeVol = matCubeL**3 # [cm^3]
# Sphere
matSphereRad = 1 # [cm]
sphereVol = (4/3)*math.pi*matSphereRad**3

# Volume Output
#Cube
fuelCubeVol = fuelVolFrac*cubeVol
graphCubeVol = graphVolFrac*cubeVol
hel_CoolCubeVol = hel_CoolVolFrac*cubeVol
totalCubeVol = fuelCubeVol + graphCubeVol + hel_CoolCubeVol
#Sphere
fuelSphereVol = fuelVolFrac*sphereVol
graphSphereVol = graphVolFrac*sphereVol
hel_CoolSphereVol = hel_CoolVolFrac*sphereVol
totalSphereVol = fuelSphereVol + graphSphereVol + hel_CoolSphereVol
# Mass Output
#Cube
fuelCubeMass = UThMixMassRho*graphCubeVol
graphCubeMass = graphMassRho*graphCubeVol
hel_CoolCubeMass = he_CoolMassRho*graphCubeVol
totalCubeMass = fuelCubeMass + graphCubeMass + hel_CoolCubeMass
#Sphere
fuelSphereMass = UThMixMassRho*graphSphereVol
graphSphereMass = graphMassRho*graphSphereVol
hel_CoolSphereMass = he_CoolMassRho*graphSphereVol
totalSphereMass = fuelSphereMass + graphSphereMass + hel_CoolSphereMass

## Define Materials

In [ ]:
fuel = openmc.Material(name='fuel');
#The below ratios were calculated assuming 95%LEU-5%Th Oxide Fuel.
fuel.add_nuclide('Th232', UThMixThAtomFrac, 'ao')
fuel.add_nuclide('U235', UThMixU235AtomFrac, 'ao') #5% U-235 enrichment
fuel.add_nuclide('U238', UThMixU238AtomFrac, 'ao')
fuel.add_element('O', UThMixOAtomFrac, 'ao')
fuel.set_density('g/cm3', UThMixMassRho) # Based on assumption of fuel density within TRISO
fuel.volume = cubeVol
print(fuel)

In [ ]:
# Establish Graphite Moderator material
graph = openmc.Material(name='graph')
# add nuclides to graph
graph.add_element('C', 1.00)
graph.set_density('g/cm3', graphMassRho)
print(graph)

In [ ]:
# Establish Helium Coolant material
hel_Cool = openmc.Material(name='hel_Cool')
# add nuclides to hel_Cool
hel_Cool.add_nuclide('He3', 0.000002)
hel_Cool.add_nuclide('He4', 0.999998)
hel_Cool.set_density('g/cm3', he_CoolMassRho)
print(hel_Cool)

In [ ]:
#materials = openmc.Materials([fuel, graph, hel_Cool])
# instead of doing this, let's try creating a material object that is a mixture
# of fuel, graphite and helium coolant

mixMat = openmc.Material.mix_materials([fuel,graph,hel_Cool],
                                       fracs=[fuelVolFrac,graphVolFrac,hel_CoolCubeVol],
                                      percent_type='vo',
                                      )

print(mixMat)



In [ ]:
materials = openmc.Materials()
#materials += [fuel, graph, hel_Cool]
materials += [mixMat]
isinstance(materials,list)

## Export Materials as XML

In [ ]:
materials.export_to_xml()
!cat materials.xml

## Define Universe Geometry

In [ ]:
l_cube = 2.0;
universeCube = openmc.model.RectangularParallelepiped(-l_cube, l_cube, -l_cube, l_cube, -l_cube, l_cube)

In [ ]:
insideCube = -universeCube
outsideCube = +universeCube

In [ ]:
cell = openmc.Cell()
cell.region = insideCube

In [ ]:
universe = openmc.Universe()
universe.add_cell(cell)

In [ ]:
universe.plot(width=(5, 5), basis='xz', colors={cell: 'fuchsia'})

## Define Bounding Geometry

In [ ]:
matBox = openmc.model.RectangularParallelepiped(-matCubeL/2, matCubeL/2, 
                                                -matCubeL/2, matCubeL/2, 
                                                -matCubeL/2, matCubeL/2,
                                                boundary_type='reflective')

In [ ]:
type(matBox)

In [ ]:
material_region = -matBox

In [ ]:
material_Geom = openmc.Cell(name='material_Geom')
#material_Geom.fill = materials #<<(SRB) This probably isn't correct.  Needs to be a material.
material_Geom.fill = mixMat
material_Geom.region = material_region

In [ ]:
root_universe = openmc.Universe(cells=[material_Geom])

geometry = openmc.Geometry()
geometry.root_universe = root_universe
geometry.export_to_xml()
!cat geometry.xml

In [ ]:
root_universe.plot(width=(2.5, 2.5), basis='xz', colors={cell: 'fuchsia'})

## Cross Sections

In [ ]:
!cat $OPENMC_CROSS_SECTIONS | head -n 10
print('    ...')
!cat $OPENMC_CROSS_SECTIONS | tail -n 10

## Source

In [ ]:
# create a point source
point = openmc.stats.Point((0,0,0))
source = openmc.Source(space=point)


In [ ]:
settings = openmc.Settings()
settings.source = source
settings.batches = 100
settings.inactive = 10
settings.particles = 1000

In [ ]:
settings.export_to_xml()
!cat settings.xml

## Tallies

In [ ]:
cell_filter = openmc.CellFilter(material_Geom)

tally = openmc.Tally(1)
tally.filters = [cell_filter]

In [ ]:
tally.nuclides = ['U235']
tally.scores = ['total','fission','absorption','(n,gamma)']

In [ ]:
tallies = openmc.Tallies([tally])
tallies.export_to_xml()
!cat tallies.xml

In [ ]:
openmc.run()

In [ ]:
!cat tallies.out

## Plot Geometry

In [ ]:
plot = openmc.Plot()
plot.filename = 'pinplot'
#plot.width = (pitch,pitch)
plot.width = (matCubeL,matCubeL)
plot.pixels = (200,200)
plot.color_by = 'material'
plot.colors = {fuel: 'yellow',hel_Cool: 'blue'}

In [ ]:
plots = openmc.Plots([plot])
plots.export_to_xml()
!cat plots.xml

In [ ]:
openmc.plot_geometry()

In [ ]:
from IPython.display import Image
Image('pinplot.png')

In [ ]:
plot.to_ipython_image()